## Capítulo 7 - Limpeza e preparaçãodos dados

Durante a análise e a modelagem dos dados, um período significativo de tempo é gasto em sua preparação: carga, limpeza, transformação e reorganização. Sabe-se que essas tarefas em geral ocupam 80% ou mais do tempo de um analista. 
    
Às vezes, o modo como os dados são armazenados em arquivos ou em bancos de dados não constituem o formato correto para uma tarefa em particular.

O pandas, junto com os recursos embutidos da linguagem Python, oferecem um conjunto de ferramentas de alto nível, rápido e flexível, para permitir que manipule os dados, deixando-os no formato correto.

Nesse capítulo, será discutido as ferramentas para dados ausentes, dados duplicados, manipulação de strings e outras transformações de dados para análise.

### 7.1 - Tratando dados ausentes

Dados ausentes são comuns em muitas aplicações de análise de dados. Um dos objetivos do pandas é deixar o trabalho com dados ausentes o menos problemático possível. Por exemplo, todas as estatísticas descritivas em objetos pandas, por padrão, excluem dados ausentes.

A forma como os dados ausentes são representados em objetos do pandas, de certo modo, não é perfeita, porém é funcional para muitos usuários. Para dados numéricos, o pandas utiliza o valor de ponto flutuante "NaN" para representá-los. Esse valor é chamado de valor sentinela, e pode ser facilmente detectado:

In [1]:
import pandas as pd
import numpy as np

In [2]:
string_data = pd.Series(['aardvark', 'artichoke', np.nan, 'avocado'])

In [3]:
string_data

0     aardvark
1    artichoke
2          NaN
3      avocado
dtype: object

In [4]:
string_data.isnull()

0    False
1    False
2     True
3    False
dtype: bool

No pandas, adotamos uma convenção usada na linguagem de programação R, referenciando, os dados ausentes como "NA", que quer dizer (Not Available) indisponível. Em aplicações estatísticas, dados NA podem ser dados inexistentes ou dados que existem, porém, não foram observados (por causa de problema com a coleta de dados, por exemplo). Ao limpar os dados para análise, em geral é importante fazer a análise nos próprios dados ausentes a fim de identificar problemas em sua coleta ou possíveis distorções provocadas por dados ausentes.

###### O valor embutido "None" de Python também é tratado como "NA" em arrays de objetos:

In [5]:
string_data[0] = None

In [6]:
string_data.isnull()

0     True
1    False
2     True
3    False
dtype: bool

Há um trabalho em andamento no projeto pandas cujo objetivo é melhorar os detalhes internos do tratamento de dados ausentes, mas as funções de API dis usuários, como "pandas.isnull", abstraem muitos dos detalhes incômodos.

Veja a tabela 7.1 que apresenta uma lista de algumas funções relacionadas ao tratamento de dados ausentes.

###### Tabela 7.1 - Métodos para tratamento de NA

Argumento

-> dropna => Filtra rótulos de eixos, baseado no fato de os valores para cada rótulo terem dados ausentes, com limites variados para a quantidade de dados ausentes a ser tolerada.

-> fillna => Preenche os dados ausentes com algum valor ou utilizando um método de interpolação como "ffill" ou "bfill".

-> isnull => Devolve valores booleanos informando quais valores estão ausentes/são NA.

-> notnull => Negação de isnull.

#### Filtrando dados ausentes

Há algumas maneiras de filtrar dados ausentes. Embora sempre haja a opção de fazer isso manualmente usando "pandas.isnull" e uma indexação booleana, o método "dropna" pode ser útil. Em uma Series, ele devolve a Series somente com os dados diferentes de null e os valores dos índices:

In [7]:
from numpy import nan as NA

In [8]:
data = pd.Series([1, NA, 3.5, NA, 7])
data

0    1.0
1    NaN
2    3.5
3    NaN
4    7.0
dtype: float64

In [9]:
data.dropna()

0    1.0
2    3.5
4    7.0
dtype: float64

###### Essa instrução abaixo, é equivalente a do dropna

In [10]:
data[data.notnull()]

0    1.0
2    3.5
4    7.0
dtype: float64

Com objetos DataFrame, a situação é um pouco mais complexa. Talvez queira descartar linhas ou colunas que conteham somente "NA" ou apenas aquelas que contenham algum "NA". Por padrão, "dropna" descarta qualquer linha contendo um valor ausente:

In [11]:
data = pd.DataFrame([[1., 6.5, 3.], [1., NA, NA],
                     [NA, NA, NA], [NA, 6.5, 3.]])
data

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [12]:
cleaned = data.dropna()
cleaned

,0,1,2
0,1.0,6.5,3.0


Passar "how=all" descartará apenas as colunas que contenham somente "NAs":

In [13]:
data.dropna(how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,NaN,6.5,3.0


Para descartar colunas do mesmo modo, passe "axis=1"

In [14]:
data[4] = NA

In [15]:
data

,0,1,2,4
0,1.0,6.5,3.0,NaN
1,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,6.5,3.0,NaN


In [16]:
data.dropna(axis=1, how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


Um modo relacionado de filtrar linhas de DataFrame diz respeito a dados de séries temporais. Suponha que queremos manter somente linhas contendo determinado número de observações. Podemos apresentar isso com o argumento "thresh"

In [17]:
df = pd.DataFrame(np.random.randn(7, 3))
df

,0,1,2
0,-0.017379,-1.533588,1.240525
1,-1.223791,-1.126998,0.185645
2,0.989479,-1.222816,-0.653302
3,1.207870,0.345858,0.184042
4,1.650927,1.341600,-1.454258
5,-0.115810,-0.025392,-0.215885
6,-1.641841,0.471972,1.252547


In [18]:
df.iloc[:4, 1] = NA

In [19]:
df.iloc[:2, 2] = NA

In [20]:
df

,0,1,2
0,-0.017379,NaN,NaN
1,-1.223791,NaN,NaN
2,0.989479,NaN,-0.653302
3,1.207870,NaN,0.184042
4,1.650927,1.341600,-1.454258
5,-0.115810,-0.025392,-0.215885
6,-1.641841,0.471972,1.252547


In [21]:
df.dropna()

,0,1,2
4,1.650927,1.341600,-1.454258
5,-0.115810,-0.025392,-0.215885
6,-1.641841,0.471972,1.252547


In [22]:
df.dropna(thresh=2)

,0,1,2
2,0.989479,NaN,-0.653302
3,1.207870,NaN,0.184042
4,1.650927,1.341600,-1.454258
5,-0.115810,-0.025392,-0.215885
6,-1.641841,0.471972,1.252547


#### Preenchendo dados ausentes

Em vez de filtrar dados ausentes (e possivelmente descartar outros dados junto com esses), poderá preencher preencher as "lacunas" de várias maneiras. Na maioria dos casos, o método "fillna" será a função que representa a força do trabalho a ser utilizada. Chamar fillna com uma constante substitui valores ausentes por esse valor:

In [23]:
df.fillna(0)

,0,1,2
0,-0.017379,0.000000,0.000000
1,-1.223791,0.000000,0.000000
2,0.989479,0.000000,-0.653302
3,1.207870,0.000000,0.184042
4,1.650927,1.341600,-1.454258
5,-0.115810,-0.025392,-0.215885
6,-1.641841,0.471972,1.252547


Ao chamar fillna com um dicionário, podemos usar um valor de preenchimento diferente para cada coluna:

In [24]:
df.fillna({1: 0.5, 2: 0})

,0,1,2
0,-0.017379,0.500000,0.000000
1,-1.223791,0.500000,0.000000
2,0.989479,0.500000,-0.653302
3,1.207870,0.500000,0.184042
4,1.650927,1.341600,-1.454258
5,-0.115810,-0.025392,-0.215885
6,-1.641841,0.471972,1.252547


fillna devolve um novo objeto, mas o objeto existente pode ser alterado in-place:

In [25]:
_ = df.fillna(0, inplace=True)

In [26]:
df

,0,1,2
0,-0.017379,0.000000,0.000000
1,-1.223791,0.000000,0.000000
2,0.989479,0.000000,-0.653302
3,1.207870,0.000000,0.184042
4,1.650927,1.341600,-1.454258
5,-0.115810,-0.025392,-0.215885
6,-1.641841,0.471972,1.252547


Os mesmos métodos de interpolação disponíveis para reindexação podem ser usados com fillna:

In [27]:
df = pd.DataFrame(np.random.randn(6, 3))
df

,0,1,2
0,-1.860147,0.936249,0.135807
1,-1.053324,-0.203469,-1.289630
2,0.979129,-1.092532,-0.356054
3,0.297664,-1.030014,-0.806628
4,1.079273,2.150292,-0.953869
5,0.123724,-1.076297,1.099611


In [28]:
df.iloc[2:, 1] = NA

In [29]:
df.iloc[4:, 2] = NA

In [30]:
df

,0,1,2
0,-1.860147,0.936249,0.135807
1,-1.053324,-0.203469,-1.289630
2,0.979129,NaN,-0.356054
3,0.297664,NaN,-0.806628
4,1.079273,NaN,NaN
5,0.123724,NaN,NaN


In [31]:
df.fillna(method='ffill')

,0,1,2
0,-1.860147,0.936249,0.135807
1,-1.053324,-0.203469,-1.289630
2,0.979129,-0.203469,-0.356054
3,0.297664,-0.203469,-0.806628
4,1.079273,-0.203469,-0.806628
5,0.123724,-0.203469,-0.806628


In [32]:
df.fillna(method='ffill', limit=2)

,0,1,2
0,-1.860147,0.936249,0.135807
1,-1.053324,-0.203469,-1.289630
2,0.979129,-0.203469,-0.356054
3,0.297664,-0.203469,-0.806628
4,1.079273,NaN,-0.806628
5,0.123724,NaN,-0.806628


Com fillna, podemos executar várias outras tarefas com um pouco de criatividade. Por exemplo, podemos passar o valor da média ou da mediana de uma Series:

In [33]:
data = pd.Series([1., NA, 3.5, NA, 7])
data

0    1.0
1    NaN
2    3.5
3    NaN
4    7.0
dtype: float64

In [34]:
data.fillna(data.mean())

0    1.000000
1    3.833333
2    3.500000
3    3.833333
4    7.000000
dtype: float64

Veja a tabela 7.2 que contém uma referência para fillna

###### Tabela 7.2 - Argumentos da função fillna

Argumento

-> value => Valor escalar ou um objeto do tipo dicionário a ser usado para preencher valores ausentes.

-> method => Interpolação; por padrão, será 'ffill' se a função for chamada sem outros argumentos.

-> axis => Eixo a ser preenchido; o default é axis = 0.

-> inplace => Modifica o objeto que faz a chamada, sem gerar uma cópia.

-> limit => Para preenchimento para a frente (forward) e para trás (backward), é o número máximo de valores consecutivos a serem preenchidos.

### 7.2 Transformação dos dados

Até agora neste capítulo, estivemos preocupados com a reorganização dos dados. Filtragem, limpeza e outras transformações constituem outra classe de operações importantes.

#### Removendo duplicatas

Linhas duplicadas podem ser encontradas em um DataFrame por diversos motivos. 

In [35]:
data = pd.DataFrame({'k1':['one', 'two'] * 3 + ['two'], 
                     'k2':[1, 1, 2, 3, 3, 4, 4]})
data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


o método "duplicated" de DataFrame devolve uma Series booleana informando se cada linha é uma duplicata (foi observada em uma linha anterior) ou não:

In [36]:
data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

Relacionado a esse caso, temos "drop_duplicates", que devolve um DataFrame com dados em que o array "duplicated" é False:

In [37]:
data.drop_duplicates()

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


Por padrão, esses dois métodos consideram todas as colunas; de forma alternativa, podemos especificar qualquer subconjunto delas na detecção de duplicatas. Suponha que tivéssemos uma coluna adicional de valores e quiséssemos filtrar as duplicatas somente com base na coluna 'k1':

In [38]:
data['v1'] = range(7)
data

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
5,two,4,5
6,two,4,6


In [39]:
data.drop_duplicates(['k1'])

,k1,k2,v1
0,one,1,0
1,two,1,1


"duplicated" e "drop_duplicates", por padrão, mantêm a primeira combinação de valores observados. Passar keep='last' devolverá a última:

In [40]:
data.drop_duplicates(['k1', 'k2'], keep='last')

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
6,two,4,6


#### Transformando dados usando uma função ou um mapeamento

Para muitos conjuntos de dados, talvez queira fazer algumas transformações com base nos valores de um array, uma Series ou uma coluna de um DataFrame. Considere os seguintes dados hipotéticos coletados acerca de vários tipos de carnes:

In [41]:
data = pd.DataFrame({'food': ['bacon', 'pulled pork', 'bacon', 
                              'Pastrami', 'corned beef', 'Bacon', 
                              'pastrami', 'honey ham', 'nova lox'], 
                     'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})

In [42]:
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


Suponha que quiséssemos adicionar uma coluna informando o tipo do animal do qual cada alimento é proveniente. Vamos criar um mapeamento entre cada tipo distinto de carne e o tipo do animal:

In [43]:
meat_to_animal = {
    'bacon': 'pig', 
    'pulled pork': 'pig', 
    'pastrami': 'cow', 
    'corned beef': 'cow', 
    'honey ham': 'pig',
    'nova lox': 'salmon'
}

O método "map" em uma Series aceita uma função ou um objeto do tipo dicionário contendo um mapeamento; nesse caso, porém temos um pequeno problema, pois algumas das carnes utilizam inicial maiúscula enquanto outras não. Desse modo, precisamos converter todos os valores para letras minúsculas utilizando o método "str.lower" de Series:

In [44]:
lowercased = data['food'].str.lower()

In [45]:
lowercased

0          bacon
1    pulled pork
2          bacon
3       pastrami
4    corned beef
5          bacon
6       pastrami
7      honey ham
8       nova lox
Name: food, dtype: object

In [46]:
data['animal'] = lowercased.map(meat_to_animal)

In [47]:
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


Poderíamos também ter passado uma função que fizesse todo o trabalho:

In [48]:
data['food'].map(lambda x: meat_to_animal[x.lower()])

0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object

Usar "map" é uma forma conveniente de fazer transformações em todos os elementos e executar outras operações relacionadas a limpeza de dados.

#### Substituindo valores

Preencher dados ausentes com o método "fillna" é um caso especial de substituição mais genérica de valores. Conforme já visto, "map" pode ser usado para modificar um subconjunto de valores em um objeto, porém "replace" oferece uma forma mais simples e mais flexível de fazer isso. 

In [49]:
data = pd.Series([1., -999., 2., -999., -1000., 3.])
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

Os valores -999 podem ser valores de sentinela para dados ausentes. Para substituí-los por valores NA, compreensíveis pelo pandas, podemos usar "replace", gerando uma nova Series (a menos que passe inplace=True).

In [50]:
data.replace(-999, np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

Se quiser substituir diversos valores de uma só vez, passe uma lista e, em seguida, o valor para substituição:

In [51]:
data.replace([-999, -1000], np.nan)

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

Para usar um substituto diferente para cada valor, passe uma lista deles:

In [52]:
data.replace([-999, -1000], [np.nan, 0])

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

O argumento especificado também pode ser um dicionário:

In [53]:
data.replace({-999: np.nan, -1000: 0})

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

#### Renomeando os índices dos eixos

Assim como os valores em uma Series, os rótulos dos eixos podem ser transformados de modo semelhante por uma função ou alguma forma de mapeamento, a fim de gerar objetos novos com rótulos diferentes. Também podemos modificar os eixos "in-place", sem criar uma nova estrutura de dados. 

In [54]:
data = pd.DataFrame(np.arange(12).reshape((3, 4)), 
                    index =['Ohio', 'Colorado', 'New York'], 
                    columns =['one', 'two', 'three', 'four'])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


Como em uma Series, os índices de um eixo têm um método map:

In [55]:
transform = lambda x: x[:4].upper()

In [56]:
data.index.map(transform)

Index(['OHIO', 'COLO', 'NEW '], dtype='object')

Podemos fazer uma atribuição para index, modificando o DataFrame in-place:

In [57]:
data.index = data.index.map(transform)

In [58]:
data

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


Se quiser criar uma versão transformada de um conjunto de dados sem modificar os dados originais, um método útil será "rename":

In [59]:
data.rename(index=str.title, columns=str.upper)

,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colo,4,5,6,7
New,8,9,10,11


Observe que rename pode ser usado em conjunto com um objeto do tipo dicionário fornecendo novos valores para um subconjunto dos rótulos de um eixo:

In [60]:
data.rename(index={'OHIO': 'INDIANA'}, 
            columns={'three': 'peekaboo'})

,one,two,peekaboo,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


"rename" evita que tenha o trabalho de copiar o DataFrame manualmente e definir seus atributos "index" e "columns". Caso queira modificar um conjunto de dados in-place, passe "inplace=True":
    

In [61]:
data.rename(index={'OHIO': 'INDIANA'}, inplace=True)

In [62]:
data

,one,two,three,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


#### Discretização e compartimentalização (binning)

Dados contínuos com frequência são discretizados ou, de modo alternativo, separados em "compartimentos" (bins) para análise. Suponha que tenhamos dados sobre um grupo de pessoas em um estudo e queremos agrupá-las em conjuntos de idades discretas:

In [63]:
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]

Vamos dividir esses dados em compartimentos de 18 a 25, 26 a 35, 36 a 60 e, por fim, 61 anos ou mais. Para isso, utilize cut, uma função do pandas:

In [64]:
bins = [18, 25, 35, 60, 100]

In [65]:
cats = pd.cut(ages, bins)
cats

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64, right]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

O objeto devolvido pelo pandas é um objeto "Categorical especial". A saída que vemos descreve os compartimentos calculados pelo "pandas.cut". Podemos tratá-la como um array de strings informando o nome do compartimento; internamente, ela contém um array "categories" que especifica os nomes distintos das categorias, junto com rótulos para os dados de "ages" no atributo codes:

In [66]:
cats.codes

array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

In [67]:
cats.categories

IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]], dtype='interval[int64, right]')

In [68]:
pd.value_counts(cats)

(18, 25]     5
(25, 35]     3
(35, 60]     3
(60, 100]    1
dtype: int64

Observe que "pd.value_counts(cats)" são os contadores de compartimentos para o resultado de "pandas.cut".

De forma consistente com a notação matemática para intervalos, um parêntese significa que o lado está "aberto", enquanto o colchete indica que está "fechado" (é inclusivo). Podemos alterar o lado que está fechado passando "right=False":

In [69]:
pd.cut(ages, [18, 26, 36, 61, 100], right=False)

[[18, 26), [18, 26), [18, 26), [26, 36), [18, 26), ..., [26, 36), [61, 100), [36, 61), [36, 61), [26, 36)]
Length: 12
Categories (4, interval[int64, left]): [[18, 26) < [26, 36) < [36, 61) < [61, 100)]

Também podemos especificar nossos próprios nomes de compartimentos passando uma lista ou um array para a opção "labels".

In [70]:
group_names = ['Youth', 'YoungAdult', 'MiddleAged', 'Senior']

In [71]:
pd.cut(ages, bins, labels=group_names)

['Youth', 'Youth', 'Youth', 'YoungAdult', 'Youth', ..., 'YoungAdult', 'Senior', 'MiddleAged', 'MiddleAged', 'YoungAdult']
Length: 12
Categories (4, object): ['Youth' < 'YoungAdult' < 'MiddleAged' < 'Senior']

Se passarmos um número inteiro de compartimentos para cut, em vez de passar fronteiras explícitas, ele calculará compartimentos de tamanhos iguais com base nos valores mínimo e máximo dos dados. Considere o caso de alguns dados uniformemente distribuídos, divididos em quartos:

In [72]:
data = np.random.rand(20)
data

array([0.37547245, 0.19804562, 0.10292682, 0.51606306, 0.35317742,
       0.6473357 , 0.77094023, 0.62878224, 0.91945724, 0.13163212,
       0.87398289, 0.10756088, 0.25718034, 0.20363275, 0.10204147,
       0.61826697, 0.24443958, 0.24741411, 0.18613319, 0.42869377])

In [73]:
pd.cut(data, 4, precision=2)

[(0.31, 0.51], (0.1, 0.31], (0.1, 0.31], (0.51, 0.72], (0.31, 0.51], ..., (0.51, 0.72], (0.1, 0.31], (0.1, 0.31], (0.1, 0.31], (0.31, 0.51]]
Length: 20
Categories (4, interval[float64, right]): [(0.1, 0.31] < (0.31, 0.51] < (0.51, 0.72] < (0.72, 0.92]]

###### A opção "precision=2" limita a precisão decimal em dois dígitos.

Uma função intimamente relacionada, "qcut", compartimenta os dados com base nos quantis da amostra. Conforme a distribuição dos dados, usar "cut" em geral não resultará em compartimento com o mesmo número de pontos de dados. 

Como "qcut" utiliza quantis da amostra, por definição, obterá compartimentos grosseiramente de mesmo tamanho:

In [74]:
data = np.random.rand(1000) # Normalmente distribuídos

In [75]:
cats = pd.qcut(data, 4) # Separa em quantis
cats

[(0.0007799999999999999, 0.24], (0.485, 0.748], (0.485, 0.748], (0.485, 0.748], (0.24, 0.485], ..., (0.748, 0.999], (0.0007799999999999999, 0.24], (0.748, 0.999], (0.485, 0.748], (0.24, 0.485]]
Length: 1000
Categories (4, interval[float64, right]): [(0.0007799999999999999, 0.24] < (0.24, 0.485] < (0.485, 0.748] < (0.748, 0.999]]

In [76]:
pd.value_counts(cats)

(0.0007799999999999999, 0.24]    250
(0.24, 0.485]                    250
(0.485, 0.748]                   250
(0.748, 0.999]                   250
dtype: int64

De modo semelhante a "cut", podemos passar nossos próprios quantis (números entre 0 e 1, inclusive):

In [77]:
pd.qcut(data, [0, 0.1, 0.5, 0.9, 1.])

[(0.0007799999999999999, 0.0994], (0.485, 0.898], (0.485, 0.898], (0.485, 0.898], (0.0994, 0.485], ..., (0.898, 0.999], (0.0994, 0.485], (0.898, 0.999], (0.485, 0.898], (0.0994, 0.485]]
Length: 1000
Categories (4, interval[float64, right]): [(0.0007799999999999999, 0.0994] < (0.0994, 0.485] < (0.485, 0.898] < (0.898, 0.999]]

#### Deetectando e filtrando valores discrepantes

Filtrar ou transformar valores discrepantes (outliers) é, em boa medida, uma questão de aplicar operações de array. Considere um DataFrame com alguns dados normalmente distribuídos:

In [78]:
data = pd.DataFrame(np.random.randn(1000, 4))

In [79]:
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.017092,0.007381,0.009465,0.042893
std,0.974247,1.046128,0.979431,1.013623
min,-3.476292,-3.241080,-3.864793,-3.051949
25%,-0.673990,-0.713762,-0.598078,-0.592108
50%,-0.010647,0.017672,0.021300,0.042634
75%,0.611029,0.714794,0.635554,0.713526
max,3.704499,3.942746,2.846989,3.432785


Suponha que quiséssemos encontrar os valorss que excedessem 3 em valor absoluto em uma das colunas:

In [80]:
col = data[2]
col

0      0.768844
1     -1.236026
2      0.067022
3     -1.079171
4     -1.663420
         ...   
995   -0.498642
996    0.839922
997    0.519436
998    0.461664
999   -1.132576
Name: 2, Length: 1000, dtype: float64

In [81]:
col[np.abs(col) > 3]

77   -3.864793
Name: 2, dtype: float64

Para selecionar todas as linhas que tenham um valor que exceda 3 ou -3, podemos utilizar o método any em um DataFrame booleano:

In [82]:
data[(np.abs(data) > 3).any(1)]

,0,1,2,3
77,1.553680,0.830918,-3.864793,-1.329870
223,-3.015837,0.018694,-0.393513,0.129546
245,-0.030844,-0.710513,-0.517549,-3.051949
518,-0.629499,1.224252,-0.830323,3.432785
550,0.840069,0.340870,-0.253512,3.217651
659,1.778017,-0.877838,-0.203303,3.110706
712,3.704499,0.084811,-0.368837,0.768971
750,-0.624781,3.942746,0.464587,0.178917
761,-0.968739,-3.241080,-0.793679,-1.658022
816,-3.476292,0.704193,0.098565,1.020616


Valores podem ser definidos com base nesses critérios. Eis um código para eliminar os valores que estejam fora do intervalo de -3 a 3:

In [83]:
data[np.abs(data) > 3] = np.sign(data) * 3
data

,0,1,2,3
0,1.074319,-0.353196,0.768844,-1.621604
1,0.607787,2.141780,-1.236026,0.314217
2,-1.657631,-1.211162,0.067022,0.228487
3,0.893736,0.014724,-1.079171,-1.596135
4,0.961818,0.848422,-1.663420,0.285173
...,...,...,...,...
995,1.350941,0.863999,-0.498642,-0.123729
996,-1.651388,-0.373465,0.839922,-0.027385
997,-0.904627,-0.908377,0.519436,-0.470919
998,-0.501691,-1.177148,0.461664,0.468662


In [84]:
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.017304,0.006451,0.010330,0.042184
std,0.970174,1.041590,0.976383,1.011118
min,-3.000000,-3.000000,-3.000000,-3.000000
25%,-0.673990,-0.713762,-0.598078,-0.592108
50%,-0.010647,0.017672,0.021300,0.042634
75%,0.611029,0.714794,0.635554,0.713526
max,3.000000,3.000000,2.846989,3.000000


A instrução "np.sign(data)" gera valores 1 e -1 com base no fato de os valores em data serem positivos ou negativos:

In [85]:
np.sign(data).head()

,0,1,2,3
0,1.0,-1.0,1.0,-1.0
1,1.0,1.0,-1.0,1.0
2,-1.0,-1.0,1.0,1.0
3,1.0,1.0,-1.0,-1.0
4,1.0,1.0,-1.0,1.0


#### Permutação e amostragem aleatória

Permutar (reordenar aleatoriamente) uma Series ou as linhas de um DataFrame é fácil utilizando a função "numpy.random.permutation". Chamar "permutation" com o tamanho do eixo que quer permutar gera um array de inteiros informando a nova ordem:

In [86]:
df = pd.DataFrame(np.arange(5 * 4).reshape((5, 4)))
df

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


In [87]:
sampler = np.random.permutation(5)
sampler

array([1, 0, 2, 4, 3])

Esse array pode então ser usado na indexação baseada em iloc ou na função "take" equivalente:

In [88]:
df

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


In [89]:
df.take(sampler)

,0,1,2,3
1,4,5,6,7
0,0,1,2,3
2,8,9,10,11
4,16,17,18,19
3,12,13,14,15


Para selecionar um subconjunto aleatório sem substituição, o método sample pode ser usado em Series e em DataFrame:

In [90]:
df.sample(n=3)

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11


Para gerar uma amostra com substituição (a fim de permitir opções repetidas), passe replace=True para sample:

In [91]:
choices = pd.Series([5, 7, -1, 6, 4])
choices

0    5
1    7
2   -1
3    6
4    4
dtype: int64

In [92]:
draws = choices.sample(n=10, replace=True)
draws

4    4
0    5
0    5
4    4
0    5
1    7
2   -1
3    6
2   -1
2   -1
dtype: int64

#### Calculando variáveis indicadoras/dummy

Outro tipo de transformação para modelagem estatística ou aplicações de aprendizado de máquina consiste em converter uma variável de categorias em uma matriz "dummy" ou "indicadora". Se uma coluna em um DataFrame tiver k valores distintos, poderíamos derivar uma matriz ou um DataFrame com k colunas contendo somente 1s e 0s. O pandas tem uma função "get_dummies" para isso, embora criar uma função por conta própria não seria difícil. Vamos retornar um exemplo anterior com DataFrame:

In [93]:
df = pd.DataFrame({'key':['b', 'b', 'a', 'c', 'a', 'b'], 
                  'data1': range(6)})
df

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [94]:
pd.get_dummies(df['key'])

,a,b,c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


Em alguns casos, talvez queira adicionar um prefixo às colunas no DataFrame indicador, que poderá então ser mesclado com os outros dados. "get_dummies" tem um argumento de prefixo para isso:

In [95]:
dummies = pd.get_dummies(df['key'], prefix='key')
dummies

,key_a,key_b,key_c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


In [96]:
df_with_dummy = df[['data1']].join(dummies)
df_with_dummy

,data1,key_a,key_b,key_c
0,0,0,1,0
1,1,0,1,0
2,2,1,0,0
3,3,0,0,1
4,4,1,0,0
5,5,0,1,0


Se uma linha de um DataFrame pertencer a várias categorias, a situação se torna um pouco mais complicada. Vamos observar o conjunto de dados de MovieLens 1M.

In [97]:
mnames = ['movie_id', 'title', 'genres']

In [98]:
movies = pd.read_table('datasets/movielens/movies.dat', sep='::', header=None, names=mnames)
movies

/tmp/ipykernel_5455/3907492227.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  movies = pd.read_table('datasets/movielens/movies.dat', sep='::', header=None, names=mnames)


,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama


In [99]:
movies[:10]

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


Adicionar variáveis indicadoras para cada gênero exige um pouco de manipulação nos dados. Inicialmente extraímos a lista de gêneros únicos do conjunto de dados:

In [100]:
all_genres = []

for x in movies.genres:
    all_genres.extend(x.split('|'))

In [101]:
genres = pd.unique(all_genres)
genres

array(['Animation', "Children's", 'Comedy', 'Adventure', 'Fantasy',
       'Romance', 'Drama', 'Action', 'Crime', 'Thriller', 'Horror',
       'Sci-Fi', 'Documentary', 'War', 'Musical', 'Mystery', 'Film-Noir',
       'Western'], dtype=object)

Uma maneira de construir o DataFrame indicador é começar com um DataFrame contendo apenas zeros:

In [102]:
zero_matrix = np.zeros((len(movies), len(genres)))
zero_matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [103]:
dummies = pd.DataFrame(zero_matrix, columns=genres)
dummies

,Animation,Children's,Comedy,Adventure,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Sci-Fi,Documentary,War,Musical,Mystery,Film-Noir,Western
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3878,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3879,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3880,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3881,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Agora itere pelos filmes e defina entradas para cada linha de "dummies" com 1. Para isso, utilize "dummies.columns" a fim de calcular os índices das colunas para cada gênero:

In [104]:
gen = movies.genres[0]
gen

"Animation|Children's|Comedy"

In [105]:
gen.split('|')

['Animation', "Children's", 'Comedy']

In [106]:
dummies.columns.get_indexer(gen.split('|'))

array([0, 1, 2])

Então podemos usar ".iloc" para definir valores com base nesses índices:

In [107]:
for i, gen in enumerate(movies.genres):
    indices = dummies.columns.get_indexer(gen.split('|'))
    dummies.iloc[i, indices] = 1

Em seguidam como fizemos antes, podemos combinar esses dados com movies:

In [108]:
movies_windic = movies.join(dummies.add_prefix('Genre_'))
movies_windic

,movie_id,title,genres,Genre_Animation,Genre_Children's,Genre_Comedy,Genre_Adventure,Genre_Fantasy,Genre_Romance,Genre_Drama,...,Genre_Crime,Genre_Thriller,Genre_Horror,Genre_Sci-Fi,Genre_Documentary,Genre_War,Genre_Musical,Genre_Mystery,Genre_Film-Noir,Genre_Western
0,1,Toy Story (1995),Animation|Children's|Comedy,1.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji (1995),Adventure|Children's|Fantasy,0.0,1.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,Grumpier Old Men (1995),Comedy|Romance,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Waiting to Exhale (1995),Comedy|Drama,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Father of the Bride Part II (1995),Comedy,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3878,3948,Meet the Parents (2000),Comedy,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3879,3949,Requiem for a Dream (2000),Drama,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3880,3950,Tigerland (2000),Drama,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3881,3951,Two Family House (2000),Drama,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [109]:
movies_windic.iloc[0]

movie_id                                       1
title                           Toy Story (1995)
genres               Animation|Children's|Comedy
Genre_Animation                              1.0
Genre_Children's                             1.0
Genre_Comedy                                 1.0
Genre_Adventure                              0.0
Genre_Fantasy                                0.0
Genre_Romance                                0.0
Genre_Drama                                  0.0
Genre_Action                                 0.0
Genre_Crime                                  0.0
Genre_Thriller                               0.0
Genre_Horror                                 0.0
Genre_Sci-Fi                                 0.0
Genre_Documentary                            0.0
Genre_War                                    0.0
Genre_Musical                                0.0
Genre_Mystery                                0.0
Genre_Film-Noir                              0.0
Genre_Western       

Uma receita útil para aplicações estatísticas é combinar "get_dummies" com uma função de discretização como o cut:

In [110]:
np.random.rand(10)

array([0.94620067, 0.09046272, 0.23593033, 0.45375309, 0.61972778,
       0.82507672, 0.89568704, 0.45548936, 0.67211204, 0.20086253])

In [111]:
values = np.random.rand(10)
values

array([0.13694807, 0.3272446 , 0.7048772 , 0.03675296, 0.16493558,
       0.13990017, 0.53392515, 0.74732216, 0.74116207, 0.47491201])

In [112]:
bins = [0, 0.2, 0.4, 0.6, 0.8, 1]

In [113]:
pd.get_dummies(pd.cut(values, bins))

,"(0.0, 0.2]","(0.2, 0.4]","(0.4, 0.6]","(0.6, 0.8]","(0.8, 1.0]"
0,1,0,0,0,0
1,0,1,0,0,0
2,0,0,0,1,0
3,1,0,0,0,0
4,1,0,0,0,0
5,1,0,0,0,0
6,0,0,1,0,0
7,0,0,0,1,0
8,0,0,0,1,0
9,0,0,1,0,0


Definimos a semente(seed) de aleatoriedade com "numpy.random.seed" para deixar o exemplo deterministico.

### 7.3 Manipulação de Strings

Python tem sido uma linguagem popular para manipulação de dados brutos há muito tempo, em parte por causa de sua facilidade de uso para processamento de strings e de texto. A maior parte das operações em texto são simplificadas com os métodos embutidos do objeto string. Para uma correspondência de padrões e manipulações de texto mais complexas, o uso de expressões regulares talvez seja necessário. O pandas complementa a mistura permitindo aplicar strings e expressões regulares de forma concisa em arrays inteiros de dados, além de lidar com transtorno dos dados ausentes.

#### Métodos de objetos string

Para muitas aplicações de manipulação de strings e de scripting, os métodos embutidos de string são suficientes. Como exemplo, uma string separada por vŕigulas pode ser dividida em partes usando "split":

In [114]:
val = 'a,b,   guido'

In [115]:
val.split(',')

['a', 'b', '   guido']

Com frequência, "split" é usado em conjunto com "strip" para remover espaços em branco (incluindo quebras de linha):


In [116]:
pieces = [x.strip() for x in val.split(',')]
pieces

['a', 'b', 'guido']

Essas substrings poderiam ser concatenadas com um delimitador de dois-pontos duplos usando a adição:

In [117]:
first, second, third = pieces

In [118]:
first + '::' + second + '::' + third

'a::b::guido'

No entanto, esse não é um método genérico prático. Uma forma mais rápida e pythônica consiste em passar uma lista ou uma tupla para o método "join" na string '::':

In [119]:
'::'.join(pieces)

'a::b::guido'

Outros métodos dizem respeito à localização de substrings. Usar a palavra reservada in de Python é a melhor maneira de detectar uma substring, embora index e find também possam ser utilizados:

In [120]:
'guido' in val

True

In [121]:
val.index(',')

1

In [122]:
val.find(':')

-1

Observe que a diferença entre find e index é que index lança uma exceção caso a string não seja encontrada (versus devolver -1):

In [123]:
val.index(':')

ValueError: substring not found

Relacionado a esse caso, temos count, que devolve o número de ocorrências de uma substring em particular:

In [124]:
val.count(',')

2

"replace" substituirá as ocorrências de um padrão por outro. É comumente utilizado também para apagar padrões, passando uma string vazia:

In [125]:
val.replace(',','::')

'a::b::   guido'

In [126]:
val.replace(',','')

'ab   guido'

Veja a tabela 7.3 que contém uma lista de alguns métodos de string de Python.

Expressões regulares também podem ser usadas com muitas dessas operações.

###### Tabela 7.3 - Métodos embutidos de strings em Python

Argumento

-> count => Devolve o número de ocorrências de uma substring na string, sem sobreposição.

-> endswith => Devolve True se a string terminar com o sufixo.

-> startswith => Devolve True se a string começar com o prefixo.

-> join => Utiliza a string como delimitadora para concatenar uma sequência de outras strings.

-> index => Devolve a posição do primeiro caracter de uma substring, se ela for encontrada em uma string: gera ValueError se não encontrar.
    
-> find => Devolve a posição do primeiro caractere da primeira ocorrência da substring na string; é como index, porém devolve -1 se não encontrar.

-> rfind => Devolve a posição do primeiro caractere da última ocorrência da substring na string; devolve -1 se não encontrar.

-> replace => Substitui ocorrências de uma string por outra string.

-> strip, rstrip, lstrip => Remove espaços em branco, incluindo quebras de linha; é equivalente a x.strip() (e a rstrip e lstrip, respectivamente) para cada elemento.

-> split => Separa a string em uma lista de substrings usando o delimitador especificado.

-> lower => Converte os caracteres alfabéticos para letras minúsculas.

-> upper => Converte os caracteres alfabéticos para letras maiúsculas.

-> casefold => Converte os caracteres para letras minúsculas e converte quaisquer combinações variáveis de caracteres específicos de região para um formato comum comparável.

-> ljust, rjust => Justifica à esquerda ou à direita, respectivamente; preenche os lado oposto da string com espaços (ou com caractere de preenchimento) para devolver uma string com um tamanho mínimo.

#### Expressões regulares

As expressões regulares oferecem uma maneira flexível para fazer pesquisas ou correspondências (em geral, mais complexas) de padrões de string em um texto. 

Uma única expressão, em geral chamada de "regex", é uma string composta de acordo com a linguagem da expressão regular. O módulo embutido "re" de Python é responsável pela aplicação de expressões regulares em strings; apresentarei uma série de exemplos de seu uso nesta seção.

As funções do módulo "re" se enquadram em três categorias: correspondência, substituição e separação de padrões. Naturalmente, elas estão relacionadas; uma regex descreve um padrão a ser localizado no texto, que pode então ser usado para vários propósitos. Vamos analisar um exemplo simples: suponha que quiséssemos separar uma string com um número variável de caracteres de espaços em branco (tabulações, espaços e quebras de linha). A regex que descreve um ou mais caracteres para espaços em branco é "\s+":

In [127]:
import re 

In [128]:
text = "foo       bar\t baz    \tqux"

In [129]:
re.split('\s+', text)

['foo', 'bar', 'baz', 'qux']

Quando chamamos "re.split('\s+', text)", a expressão regular inicialmente é compilada; então seu método split é chamado no texto que lhe é passado. Podemos compilar a regex por conta própria com "re.compile", criando um objeto regex reutilizável:

In [130]:
regex = re.compile('\s+')

In [131]:
regex.split(text)

['foo', 'bar', 'baz', 'qux']

Se, em vez disso, quiséssemos obter uma lista de todos os padrões que correspondam à regex, o método "findall" poderia ser utilizado:

In [132]:
regex.findall(text)

['       ', '\t ', '    \t']

Criar um objeto regex com re.compile é altamente recomendado caso pretenda aplicar a mesma expressão a várias strings; fazer isso economizará ciclos de CPU.

"match" e "search" estão intimamente relacionados a "findall". Enquando "findall" devolve todas as correspondências em uma string, "search" devolve apenas a primeira. De modo mais rigoroso, "match" faz a correspondência somente no início da string. Como um exemplo menos trivial, vamos considerar um bloco de texto e uma expressão regular capaz de identificar a maioria dos endereços de email:

In [133]:
text = """Dave dave@google.com
Steve steve@gmail.com
Rob rob@gmail.com
Ryan ryan@google.com
"""

In [134]:
pattern = r'[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,4}'

In [135]:
# re.IGNORECASE faz com a regex não diferencie letras minúsculas de maiúsculas
regex = re.compile(pattern, flags=re.IGNORECASE)

Usar "findall" no texto gera uma lista de endereços de email:

In [136]:
regex.findall(text)

['dave@google.com', 'steve@gmail.com', 'rob@gmail.com', 'ryan@google.com']

"search" devolve um objeto especial de correspondência para o primeiro endereço de email de texto. Para regex anterior, o objeto de correspondência pode nos dizer apenas a posição de início e de fim do padrão na string:

In [137]:
m = regex.search(text)
m

<re.Match object; span=(5, 20), match='dave@google.com'>

In [138]:
text[m.start():m.end()]

'dave@google.com'

"regex.match" devolve None, pois fará a correspondência somente se o padrão ocorrer no início da string:

In [139]:
print(regex.match(text))

None


Relacionado a esse caso, temos "sub", que devolverá uma nova string com as ocorrências do padrão substituídas por uma nova string:

In [140]:
print(regex.sub('REDACTED', text))

Dave REDACTED
Steve REDACTED
Rob REDACTED
Ryan REDACTED



Suponha que quiséssemos encontrar os endereços de email e, simultaneamente, segmentar cada endereço em seus três componentes: nome do usuário, nome do domínio e sufixo do domínio. Para isso, coloque parênteses em torno das partes padrão a fim de segmentá-lo:

In [141]:
pattern = r'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})'

In [142]:
regex = re.compile(pattern, flags=re.IGNORECASE)

Um objeto de correspondência gerado por essa regex modificada devolve uma tupla dos componentes do padrão com seu método "groups":

In [143]:
m = regex.match('wesm@bright.net')

In [144]:
m.groups()

('wesm', 'bright', 'net')

"findall" devolve uma lista de tuplas se o padrão tiver grupos:

In [145]:
regex.findall(text)

[('dave', 'google', 'com'),
 ('steve', 'gmail', 'com'),
 ('rob', 'gmail', 'com'),
 ('ryan', 'google', 'com')]

"sub" também tem acesso aos grupos em cada correspondência, usando símbolos especiais como \1 e \2. O símbolo \1 corresponde ao primeiro grupo identificado, \2 corresponde ao segundo grupo, e assim sucessivamente:

In [146]:
print(regex.sub(r'Username: \1, Domain: \2, Sufix: \3', text))

Dave Username: dave, Domain: google, Sufix: com
Steve Username: steve, Domain: gmail, Sufix: com
Rob Username: rob, Domain: gmail, Sufix: com
Ryan Username: ryan, Domain: google, Sufix: com



Há muito mais informações sobre expressões regulares em Python

###### Tabela 7.4 - Métodos de expressões regulares

Argumento 

-> findall => Devolve todos os padrões correspondentes em uma string, sem sobreposição, na forma de uma lista.

-> finditer => é como "findall", porém devolve um iterador.

-> match => Corresponde padrões no início da string e, opcionalmente, segmenta componentes do padrão em grupos; se houver uma correspondência com o padrão, devolve um objeto de correspondência; caso contrário, devolve None.

-> search => Pesquisa a string para verificar se há uma correspondência com o padrão; em caso afirmativo, devolve um objeto de correspondência. De modo diferente de "match", a correspondência pode se dar em qualquer ponto da string, em oposição a ocorrer somente no início.

-> split => Separa a string em partes a cada ocorrência do padrão.

-> sub, subn => Substitui todas (sub) ou as n primeiras (subn) ocorrẽncias do padrão em uma string por uma expressão substituta; utiliza os símbolos \1, \2, ... para referenciar os elementos de grupo da correspondência na string de substituição.

#### Funções de string vetorizadas no pandas

Limpar um conjunto de dados desorganizados para análise em geral exige muita manipulação e regularização de strings. Para complicar mais ainda a situação, uma coluna contendo strings ocasionalmente terá dados ausentes:

In [147]:
data = {'Dave': 'dave@google.com', 'Steve': 'steve@gmail.com', 'Rob': 'rob@gmail.com', 'Wes': np.nan}

In [148]:
data = pd.Series(data)
data

Dave     dave@google.com
Steve    steve@gmail.com
Rob        rob@gmail.com
Wes                  NaN
dtype: object

In [149]:
data.isnull()

Dave     False
Steve    False
Rob      False
Wes       True
dtype: bool

Podemos aplicar métodos de string e de expressões regulares (passando uma lambda ou outra função) para cada valor usando "data.map", mas haverá uma falha em valores NA (nulos). 
Para lidar com isso, Series tem métodos orientados a arrays para operações em string, que ignoram valores NA. Eles são acessados por meio do atributo "str" de Series; por exemplo, poderíamos verificar se cada endereço de email contém "gmail" usando "str.contains".

In [150]:
data.str.contains('gmail')

Dave     False
Steve     True
Rob       True
Wes        NaN
dtype: object

Expressões regulares também podem ser usadas, junto com qualquer opção de "re", como IGNORECASE.

In [151]:
pattern

'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\\.([A-Z]{2,4})'

In [152]:
data.str.findall(pattern, flags=re.IGNORECASE)

Dave     [(dave, google, com)]
Steve    [(steve, gmail, com)]
Rob        [(rob, gmail, com)]
Wes                        NaN
dtype: object

Há duas maneiras para a obtenção de elementos vetorizados. Pode usar "str.get" ou indexar no atributo "str":

In [156]:
matches = data.str.findall(pattern, flags=re.IGNORECASE).str[0]
matches

Dave     (dave, google, com)
Steve    (steve, gmail, com)
Rob        (rob, gmail, com)
Wes                      NaN
dtype: object

Para acessar os elementos nas listas embutidas, podemos passar um índice para qualquer uma dessas funções:

In [157]:
matches.str.get(1)

Dave     google
Steve     gmail
Rob       gmail
Wes         NaN
dtype: object

In [158]:
matches.str[0]

Dave      dave
Steve    steve
Rob        rob
Wes        NaN
dtype: object

De modo semelhante, podemos fatiar as strings usando a sintaxe a seguir: 

In [160]:
data.str[:5]

Dave     dave@
Steve    steve
Rob      rob@g
Wes        NaN
dtype: object

Veja a tabela 7.5 que apresenta outros métodos de string do pandas.

###### Tabela 7.5 - Listagem parcial dos métodos de string vetorizados

Método 

-> cat => Concatena strings em todos os elementos, com um delimitador opcional.

-> contains => Devolve um array booleano se cada string contiver um padrão/uma regex

-> count => Conta as ocorrências do padrão.

-> extract => Utiliza uma expressão regular com grupos para extrair uma ou mais strings de uma Series de strings; o resultado será um DataFrame com uma coluna por grupo.

-> endswith => Equivalente a x.endswith(pattern) para cada elemento.

-> startwith => Equivalente a x.startswith(pattern) para cada elemento.

-> findall => Calcula uma lista com todas as ocorrências de um padrão/uma regex para cada string. 

-> get => Indexa cada elemento (obtém o i-ésimo elemento).

-> isalnum => Equivalente ao str.alnum embutido.

-> isalpha => Equivalente ao str.isalpha embutido.

-> isdecimal => Equivalente ao str.isdecimal embutido.

-> isdigit => Equivalente ao str.isdigit embutido.

-> islower => Equivalente ao str.islower embutido.

-> isnumeric => Equivalente ao str.isnumeric embutido.

-> isuper => Equivalente ao str.isuper embutido.

-> join => Junta strings em cada elemento da Series utilizando o separador especificado.

-> len => Calcula o tamanho de cada string.

-> lower, upper => Converte para letras minúsculas ou maiúsculas; equivalente a x.lower() ou a x.upper() para cada elemento.

-> match => Usa re.match com a expressão regular especificadas em cada elemento, devolvendo os grupos com os quais houve uma correspondência, na forma de uma lista.

-> pad => Adiciona espaços em branco à esquerda, à direita ou nos dois lados das strings.

-> center => Equivalente a pad(side='both').

-> repeat => Duplica valores (por exemplo, s.str.repeat(3) é equivalente a x * 3 para cada string).

-> replace => Substitui ocorrências do padrão/da regex por outra string.

-> slice => Fatia cada string da Series.

-> split => Separa as string no delimitador ou na expressão regular.

-> strip => remove espaços em branco em ambos os lados, incluindo quebras de linha.

-> strip => Remove espaços em branco do lado direito.

-> lstrip => Remove espaços em branco do lado esquerdo.

### 7.4 Conclusão

Uma preparação de dados eficiente pode melhorar de modo significativo a produtividade, permitindo que invista mais tempo analisando dados e menos tempo preparando-os para a análise. Exploramos uma série de ferramentas neste capítulo, mas a abrangência, de forma alguma, foi completa.